In [1]:
# coco_vg_info: list of info for images that are in vg and coco
import json
from pprint import pprint
with open('coco_overlap.json') as data_file:
    coco_vg_info = json.load(data_file)
print(len(coco_vg_info))

51498


In [2]:
# vg_idx: vg image that also in coco
vg_idx = []
for dic in coco_vg_info:
    vg_idx.append(int(dic['image_id']))
vg_idx.sort()
print(len(vg_idx))                     

51498


In [27]:
print(vg_idx[0])

61512


# h5 format
> **key: gt**   
>> group key <HDF5 group "/gt/test" (25858 members)> <br/>
>> group key <HDF5 group "/gt/train" (73794 members)> <br/>

> **key: meta**
>> group key <HDF5 group "/meta/cls" (2 members)> <br/>
>>> idx2name
>>> name2idx
>>group key <HDF5 group "/meta/imid2path" (108077 members)> <br/>
>> group key <HDF5 group "/meta/pre" (2 members)> <br/>


In [38]:
import h5py
import os
import bisect

In [35]:
path=['/gt/test/', '/gt/train/']
test = 0
train = 0
print(os.path.getsize('vg1_2_meta2.h5'))

278024180


In [41]:
deleted=0
with h5py.File('vg1_2_meta2.h5','a') as f:
    for key in f[path[0]].keys():
        test +=1
        int_key = int(key)
        # test whether key is in vg_idx
        idx = bisect.bisect(vg_idx, int_key)
        if vg_idx[idx-1]!=int_key: # not in vg_idx
            # delete that group
            del f[path[0]+str(key)]
            deleted = deleted + 1
#         if key not in vg_idx:
#             del f[path[0]+str(key)]
#         f.__delitem__('gt/train/2391709')
#         print(f['gt/train/2391709'])
print(deleted)

13473


In [43]:
deleted=0
with h5py.File('vg1_2_meta2.h5','a') as f:
    for key in f[path[1]].keys():
        test +=1
        int_key = int(key)
        # test whether key is in vg_idx
        idx = bisect.bisect(vg_idx, int_key)
        if vg_idx[idx-1]!=int_key: # not in vg_idx
            # delete that group
            del f[path[1]+str(key)]
            deleted = deleted + 1
#         if key not in vg_idx:
#             del f[path[0]+str(key)]
#         f.__delitem__('gt/train/2391709')
#         print(f['gt/train/2391709'])
print(deleted)

38732


In [48]:
f.close()
print(os.path.getsize('vg1_2_meta2.h5'))

278024180


In [55]:
paths=['/gt/test','/gt/train', '/meta/cls','/meta/imid2path','/meta/pre']
fs = h5py.File('vg1_2_meta2.h5', 'r')
fd = h5py.File('cleaned_vg.h5','w')
for path in paths:
    group_path = fs[path].parent.name
    group_id = fd.require_group(group_path)
    fs.copy(path, group_id, name=path.split('/')[-1])

In [57]:
for path in paths:
    print(fd[path])

<HDF5 group "/gt/test" (12385 members)>
<HDF5 group "/gt/train" (35062 members)>
<HDF5 group "/meta/cls" (2 members)>
<HDF5 group "/meta/imid2path" (108077 members)>
<HDF5 group "/meta/pre" (2 members)>


In [58]:
fs.close()
fd.close()